In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 86.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 27.2 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import torch
from transformers import DebertaTokenizer, DebertaModel

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/text_classification/dot/word2vec-deberta

/content/drive/MyDrive/text_classification/dot/word2vec-deberta


In [ ]:
## deberta ##
# 读取数据
data = pd.read_excel('./data.xlsx')
sentences = data['review'].tolist()

tokenizer = DebertaTokenizer.from_pretrained('microsoft/deberta-base')
model = DebertaModel.from_pretrained('microsoft/deberta-base')

word_vectors = []

# 遍历每个句子并获取单词的词向量
for sentence in sentences:
    # 使用 tokenizer 对句子进行分词
    tokens = tokenizer.tokenize(sentence)

    # 遍历分词后的 tokens 并获取每个单词的词向量
    for token in tokens:
        # 将单词 token 传递给模型以获取词向量
        inputs = tokenizer(token, return_tensors='pt', padding=True, truncation=True)
        with torch.no_grad():
            outputs = model(**inputs)
            word_vector = outputs.last_hidden_state.squeeze(0).mean(dim=0).cpu().numpy()  # 获取词向量并取平均
            word_vectors.append((token, word_vector))

In [ ]:
# # 指定的单词
# word = '宗教'

# # 使用 tokenizer 对单词进行编码
# inputs = tokenizer(word, return_tensors='pt', padding=True, truncation=True)

# # 传递编码后的输入到 DeBERTa 模型以获取词向量
# with torch.no_grad():
#     outputs = model(**inputs)

# # 获取单词 "宗教" 的词向量
# word_vector = outputs.last_hidden_state.squeeze(0).mean(dim=0).cpu().numpy()

# # 打印单词 "宗教" 的词向量
# print(f"Word: {word}, Word Vector: {word_vector}")

In [46]:
import pandas as pd
from gensim.models import Word2Vec
import jieba

In [ ]:
## word2vec ##
def txt_cut(s):
    res = [w for w in jieba.lcut(s) if w.strip()]
    return " ".join(res)

# 对文本进行分词
tokenized_text = []
for i in sentences:
    new = txt_cut(i).split(' ')
    tokenized_text.extend(new)

# 初始化Word2Vec模型
model_w = Word2Vec(vector_size=100, window=5, min_count=1)

# 构建词汇表
model_w.build_vocab([tokenized_text])

# 训练模型
model_w.train([tokenized_text], total_examples=model_w.corpus_count, epochs=model_w.epochs)

# 创建一个空的DataFrame来保存词汇和对应的词向量
word_vector_df = pd.DataFrame()

for wd in model_w.wv.index_to_key:
    # 将词向量转化为Series，然后添加到DataFrame中
    word_vector_series = pd.Series([wd] + list(model_w.wv[wd]))
    word_vector_df = word_vector_df.append(word_vector_series, ignore_index=True)

In [ ]:
# word = "宗教"
# word_vector_2 = model_w.wv[word]
# print(word_vector_2)

In [53]:
new_word_vectors = []

for word in tokenized_text:
    ## deberta
    # 使用 tokenizer 对单词进行编码
    inputs = tokenizer(word, return_tensors='pt', padding=True, truncation=True)

    # 传递编码后的输入到 DeBERTa 模型以获取词向量
    with torch.no_grad():
        outputs = model(**inputs)

    # 获取单词 "宗教" 的词向量
    word_vector_d = outputs.last_hidden_state.squeeze(0).mean(dim=0).cpu().numpy()

    word_vector_d = word_vector_d.reshape(-1, 1)

    ## word2vec
    word_vector_w = model_w.wv[word]

    word_vector_w = word_vector_w.reshape(-1, 1)

    ## 新词向量
    new_word_vector = np.multiply(word_vector_d, word_vector_w.T)

    new_word_vectors.append((word, new_word_vector))

In [61]:
# 指定保存文件路径
save_path = 'new_word_vectors.txt'

# 保存单词和新词向量到文件
with open(save_path, 'w') as f:
    for word, word_vector in new_word_vectors:
        word_vector_str = ','.join(map(str, word_vector))
        line = f"{word}: {word_vector_str}\n"
        f.write(line)